# City and Weather API's

## City API

In [1]:
import pandas as pd
import os
local_project_path = r'C:\Users\antoi\\Documents\\Work&Learn\\JEDHA\\M03-DataCollection_Managment\\JEDHA-Projet-2-Kayak\\'
path = local_project_path+'datas'

chemin_fichier_csv = path+"./city.csv"
df = pd.read_csv(chemin_fichier_csv, header=None, names=['city','nan'])
cities = df['city'].str.replace(' ', '-').tolist()

# list(cities)

In [2]:
import requests

api_url = 'https://nominatim.openstreetmap.org/search'

enriched_cities = []

for city in cities:

    payload = {
        'city' : city,
        'limit' : 1,
        'country' : 'france',
        'country-code' : 'fr',
        'format' : 'json'
        }
    
    r_city = requests.get(api_url, 
                          params=payload)

    enriched_cities.append({'city': city,
                            'lat' : float(r_city.json()[0]['lat']),
                            'lon' : float(r_city.json()[0]['lon'])})
    
# display(enriched_cities)


In [3]:
df = pd.DataFrame.from_dict(enriched_cities)

In [4]:
import pandas as pd
import plotly.express as px
fig = px.scatter_mapbox(df,
                        lat='lat',
                        lon='lon',
                        center=dict(lat=46.2, lon=2.2),
                        zoom=4,
                        mapbox_style='carto-positron',
                        hover_name='city',
                        height=600)

fig.show()

## Weather API

In [5]:
print(os.getcwd())


c:\Users\antoi\Documents\Work&Learn\JEDHA\M03-DataCollection_Managment\JEDHA-Projet-2-Kayak


In [6]:

with open("API_key.txt", "r") as file:
    API_key = str(file.readline())

exclude = 'current,minutely,hourly,alerts'
weather_cities = []

for city in enriched_cities:
    lat = city['lat']
    lon = city['lon']
    url = f'https://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}&exclude={exclude}&appid={API_key}&units=metric'
    weather_raw = requests.get(url).json()
    weather_raw['city']['name'] = city['city']
    weather_cities.append(weather_raw)

# display(weather_cities)

In [7]:
pd.json_normalize(weather_cities).head()

,cod,message,cnt,list,city.id,city.name,city.coord.lat,city.coord.lon,city.country,city.population,city.timezone,city.sunrise,city.sunset
0,200,0,40,"[{'dt': 1712113200, 'main': {'temp': 11.04, 'f...",6435453,Mont-Saint-Michel,48.6360,-1.5115,FR,0,7200,1712122736,1712169548
1,200,0,40,"[{'dt': 1712113200, 'main': {'temp': 11.12, 'f...",2978640,St-Malo,48.6495,-2.0260,FR,50676,7200,1712122859,1712169673
2,200,0,40,"[{'dt': 1712113200, 'main': {'temp': 10.17, 'f...",6447202,Bayeux,49.2765,-0.7025,FR,13911,7200,1712122504,1712169392
3,200,0,40,"[{'dt': 1712113200, 'main': {'temp': 10.42, 'f...",3003796,Le-Havre,49.4939,0.1080,FR,185972,7200,1712122296,1712169211
4,200,0,40,"[{'dt': 1712113200, 'main': {'temp': 10.56, 'f...",2982652,Rouen,49.4405,1.0940,FR,112787,7200,1712122063,1712168971


In [8]:
pd.json_normalize(weather_cities)['city.name']
# pd.json_normalize(weather_cities)['list']
# pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])
# pd.json_normalize(pd.json_normalize(weather_cities)['list'][34])['weather'][0]

0                Mont-Saint-Michel
1                          St-Malo
2                           Bayeux
3                         Le-Havre
4                            Rouen
5                            Paris
6                           Amiens
7                            Lille
8                       Strasbourg
9     Chateau-du-Haut-Koenigsbourg
10                          Colmar
11                       Eguisheim
12                        Besancon
13                           Dijon
14                          Annecy
15                        Grenoble
16                            Lyon
17                Gorges-du-Verdon
18              Bormes-les-Mimosas
19                          Cassis
20                       Marseille
21                 Aix-en-Provence
22                         Avignon
23                            Uzes
24                           Nimes
25                   Aigues-Mortes
26        Saintes-Maries-de-la-mer
27                       Collioure
28                  

In [9]:
weather_datas = pd.DataFrame()

for id, city in enumerate(pd.json_normalize(weather_cities)['city.name']):

    df3 = pd.json_normalize(pd.json_normalize(weather_cities)['list'][id])
    df3['city'] = city
    df3['lat'] = pd.json_normalize(weather_cities)["city.coord.lat"][id]
    df3['lon'] = pd.json_normalize(weather_cities)["city.coord.lon"][id]
    
    weather_datas = pd.concat([weather_datas, df3])

weather_datas['dt_txt'] = pd.to_datetime(weather_datas['dt_txt']) 
display(weather_datas.head(3))

,dt,weather,visibility,pop,dt_txt,main.temp,main.feels_like,main.temp_min,main.temp_max,main.pressure,...,main.temp_kf,clouds.all,wind.speed,wind.deg,wind.gust,sys.pod,rain.3h,city,lat,lon
0,1712113200,"[{'id': 804, 'main': 'Clouds', 'description': ...",10000,0.17,2024-04-03 03:00:00,11.04,10.65,10.56,11.04,1002,...,0.48,100,4.42,193,9.97,n,NaN,Mont-Saint-Michel,48.636,-1.5115
1,1712124000,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,0.20,2024-04-03 06:00:00,10.92,10.50,10.74,10.92,1003,...,0.18,99,5.78,217,11.35,d,0.13,Mont-Saint-Michel,48.636,-1.5115
2,1712134800,"[{'id': 500, 'main': 'Rain', 'description': 'l...",10000,1.00,2024-04-03 09:00:00,10.15,9.65,10.15,10.15,1007,...,0.00,100,5.33,275,10.51,d,1.33,Mont-Saint-Michel,48.636,-1.5115


In [10]:
col_drop = ['dt', 'weather', 'visibility', 'main.temp', 'main.temp_min', 'main.temp_max', 'main.pressure','main.sea_level',
            'main.grnd_level', 'main.humidity', 'main.temp_kf', 'clouds.all', 'wind.deg', 'wind.gust', 'rain.3h']

weather_datas.drop(columns=col_drop, inplace=True)


In [11]:
weather_during_day = weather_datas[weather_datas['sys.pod']=='d'].drop(columns='sys.pod')

In [12]:
weather_day_mean = weather_during_day.groupby('city').mean().reset_index()

In [13]:
display(weather_day_mean[['city','pop']].sort_values('pop').head())
display(weather_day_mean[['city','main.feels_like']].sort_values('main.feels_like',ascending=False).head())
display(weather_day_mean[['city','wind.speed']].sort_values('wind.speed').head())

,city,pop
0,Aigues-Mortes,0.0
1,Aix-en-Provence,0.0
27,Nimes,0.0
24,Marseille,0.0
14,Collioure,0.0


,city,main.feels_like
4,Ariege,18.1080
7,Bayonne,17.1804
19,Grenoble,17.0840
1,Aix-en-Provence,17.0644
14,Collioure,16.8684


,city,wind.speed
3,Annecy,1.9740
19,Grenoble,2.0332
4,Ariege,2.4212
18,Gorges-du-Verdon,2.5856
1,Aix-en-Provence,2.8984


Create a SCORE to select the city

In [14]:
weights = {'pop': -0.4, 'main.feels_like': 0.5, 'wind.speed': -0.1}

weather_day_mean['score'] = (weather_day_mean['pop'] * weights['pop'] +
                             weather_day_mean['main.feels_like'] * weights['main.feels_like'] +
                             weather_day_mean['wind.speed'] * weights['wind.speed'])

min_score = weather_day_mean['score'].min()
max_score = weather_day_mean['score'].max()
weather_day_mean['score'] = 10 * (weather_day_mean['score'] - min_score) / (max_score - min_score)

display(weather_day_mean.sort_values('score',ascending=False).head())

,city,pop,dt_txt,main.feels_like,wind.speed,lat,lon,score
4,Ariege,0.016,2024-04-05 12:00:00,18.1080,2.4212,42.8458,1.6068,10.000000
19,Grenoble,0.126,2024-04-05 12:00:00,17.0840,2.0332,45.1876,5.7358,8.548545
1,Aix-en-Provence,0.000,2024-04-05 12:00:00,17.0644,2.8984,43.5298,5.4475,8.419676
7,Bayonne,0.080,2024-04-05 12:00:00,17.1804,3.2704,43.4945,-1.4737,8.388245
14,Collioure,0.000,2024-04-05 12:00:00,16.8684,3.3384,42.5251,3.0832,8.021171


In [15]:
fig = px.scatter_mapbox(weather_day_mean,
                        lat='lat',
                        lon='lon',
                        color='score',
                        size='score',
                        center=dict(lat=47, lon=2.2),
                        zoom=4.5,
                        color_continuous_scale=['white', 'yellow','orange', 'red','purple'],
                        mapbox_style='carto-positron',
                        hover_name='city',
                        hover_data=['main.feels_like', "pop", "wind.speed", "score"],
                        height=600,
                        width=600*16/9)

fig.show()

In [16]:
top5_cities = weather_day_mean.sort_values('score',ascending=False).head()
top5_cities

,city,pop,dt_txt,main.feels_like,wind.speed,lat,lon,score
4,Ariege,0.016,2024-04-05 12:00:00,18.1080,2.4212,42.8458,1.6068,10.000000
19,Grenoble,0.126,2024-04-05 12:00:00,17.0840,2.0332,45.1876,5.7358,8.548545
1,Aix-en-Provence,0.000,2024-04-05 12:00:00,17.0644,2.8984,43.5298,5.4475,8.419676
7,Bayonne,0.080,2024-04-05 12:00:00,17.1804,3.2704,43.4945,-1.4737,8.388245
14,Collioure,0.000,2024-04-05 12:00:00,16.8684,3.3384,42.5251,3.0832,8.021171


In [21]:

fig = px.scatter_mapbox(top5_cities,
                        lat='lat',
                        lon='lon',
                        color='score',
                        size='score',
                        center=dict(lat=top5_cities['lat'].mean(), lon=top5_cities['lon'].mean()),
                        zoom=6,
                        range_color=(0, 10),
                        color_continuous_scale=['white', 'yellow','orange', 'red','purple'],
                        mapbox_style='carto-positron',
                        text='city',
                        hover_name='city',
                        hover_data=['main.feels_like', "pop", "wind.speed", "score"],
                        height=600,
                        width=600*16/9)


fig.show()

In [18]:
weather_day_mean.rename(columns={
    'city': 'city_target',
    'pop': 'city_target_precipitation',
    'dt_txt': 'date_time',
    'main.feels_like': 'city_target_temp_feels_like',
    'wind.speed': 'city_target_wind_speed',
    'lat': 'lat_target',
    'lon': 'lon_target',
    'score': 'city_target_weather_score',
}, inplace=True)

In [23]:
weather_day_mean.sort_values('city_target_weather_score',ascending=False)

,city_target,city_target_precipitation,date_time,city_target_temp_feels_like,city_target_wind_speed,lat_target,lon_target,city_target_weather_score
4,Ariege,0.0160,2024-04-05 12:00:00,18.1080,2.4212,42.8458,1.6068,10.000000
19,Grenoble,0.1260,2024-04-05 12:00:00,17.0840,2.0332,45.1876,5.7358,8.548545
1,Aix-en-Provence,0.0000,2024-04-05 12:00:00,17.0644,2.8984,43.5298,5.4475,8.419676
7,Bayonne,0.0800,2024-04-05 12:00:00,17.1804,3.2704,43.4945,-1.4737,8.388245
14,Collioure,0.0000,2024-04-05 12:00:00,16.8684,3.3384,42.5251,3.0832,8.021171
5,Avignon,0.0208,2024-04-05 12:00:00,16.4620,3.0988,43.9492,4.8059,7.494808
9,Biarritz,0.0800,2024-04-05 12:00:00,16.5572,3.4708,43.4833,-1.5593,7.457540
26,Montauban,0.0844,2024-04-05 12:00:00,16.2712,5.0772,44.0176,1.3550,6.600474
24,Marseille,0.0000,2024-04-05 12:00:00,16.0360,4.7672,43.2962,5.3700,6.452185
27,Nimes,0.0000,2024-04-05 12:00:00,15.7792,3.6264,43.8374,4.3601,6.411998


In [20]:
path=local_project_path+'datas'
weather_day_mean.to_csv('cities_weather_datas.csv',index=False)